In [91]:
import pandas as pd
from collections import Counter
from copy import copy
# Lists of given courses and TAs
TAs = ['Aya','Alaa','Mohammed','Bassam','Ali','Ahmed','Arwa',"AboBakr","Pola","Farah","Yassmine","Salwa"]
courses = ['205','207','217','479','305','307','317','379']
# Preferred courses for each TA dataframe (1 being highest and 8 lowest)
pref = pd.DataFrame({'205': [2,3,2,8,7,3,5,1,1,2,3,1], '207': [3,2,1,6,8,1,7,5,4,5,6,8], '217':[1,1,4,5,6,4,8,2,7,8,1,2], '479':[4,4,3,7,5,2,6,8,2,3,4,7],'305': [5,6,7,1,2,8,3,4,5,6,7,3], '307': [6,5,8,4,1,7,1,3,8,1,2,6], '317':[7,8,6,2,4,5,4,6,3,4,5,4], '379':[8,7,5,3,3,6,2,7,6,7,8,5]})
pref.index = TAs
pref

,205,207,217,479,305,307,317,379
Aya,2,3,1,4,5,6,7,8
Alaa,3,2,1,4,6,5,8,7
Mohammed,2,1,4,3,7,8,6,5
Bassam,8,6,5,7,1,4,2,3
Ali,7,8,6,5,2,1,4,3
Ahmed,3,1,4,2,8,7,5,6
Arwa,5,7,8,6,3,1,4,2
AboBakr,1,5,2,8,4,3,6,7
Pola,1,4,7,2,5,8,3,6
Farah,2,5,8,3,6,1,4,7


In [92]:
# Skilled courses for each TA dataframe (1 being highest and 8 lowest)
skill = pd.DataFrame({'205': [8,6,5,1,2,4,3,6,7,8,8,1], '207': [7,8,6,4,1,3,1,3,4,5,7,2], '217':[6,5,7,2,4,1,4,8,1,2,6,3], '479':[5,7,8,3,3,2,2,5,6,7,3,4],'305': [3,3,2,5,8,7,6,2,3,4,4,8], '307': [2,2,1,6,7,5,8,7,8,1,2,5], '317':[1,1,4,7,6,8,5,4,5,6,5,6], '379':[4,4,3,8,5,6,7,1,2,3,1,7]})
skill.index = TAs
skill

,205,207,217,479,305,307,317,379
Aya,8,7,6,5,3,2,1,4
Alaa,6,8,5,7,3,2,1,4
Mohammed,5,6,7,8,2,1,4,3
Bassam,1,4,2,3,5,6,7,8
Ali,2,1,4,3,8,7,6,5
Ahmed,4,3,1,2,7,5,8,6
Arwa,3,1,4,2,6,8,5,7
AboBakr,6,3,8,5,2,7,4,1
Pola,7,4,1,6,3,8,5,2
Farah,8,5,2,7,4,1,6,3


In [93]:
# dict to control which course each TA can assign to
courses_available = {ta:courses for ta in TAs}
# waiting list of TAs that were assigned on each iteration
waiting_list = []
# dict to store created assigning
assigned = {}
# variable to count number of iterations
count = 0

In [94]:
# while not all TAs have been assigned
while len(waiting_list)<(len(TAs)):
    # TA is being assigned
    for ta in TAs:
        # each ta is assigned to it's preferred course from it's list
        if ta not in waiting_list:
            #List of courses
            co = courses_available[ta]
            # Return course with most preferred for a specific TA
            best_choice = pref.loc[ta][pref.loc[ta].index.isin(co)].idxmin()
            # Add to the dict the key(TA, course assigned) with value (rank in preferred, rank in skilled)
            assigned[(ta, best_choice)]=(pref.loc[ta][best_choice],
                                                 skill.loc[ta][best_choice])                                                                              

    """
        If one course has more than one assigning 
        it will choose the skilled option
    """
    # return count of each assigned course
    countt = Counter([key[1] for key in assigned.keys()])

    # cycle to choose the best options
    for c in countt.keys():
        if countt[c] in range(1,2):
            #return list of tuples after being sorted with their skilled rank and then remove the first tuple [0]
            # as being best choice to assign
            to_drop = sorted({pair: assigned[pair] for pair in assigned.keys() 
                    if c in pair}.items(), 
                   key=lambda x: x[1][1])[1:]

            for p_to_drop in to_drop:
                #delete the to_drop repeated assigning from assigned dict if it has least skilled compared to the preferred
                if (p_to_drop[1][1])>4:
                    del assigned[p_to_drop[0]]
                
                # copy the available courses for this TA
                    cour = copy(courses_available[p_to_drop[0][0]])
                # remove the repeated course from the TA's available courses
                    cour.remove(p_to_drop[0][1])
                # after removing the declined course, switch the copied available course and the original
                    courses_available[p_to_drop[0][0]] = cour
            
                
    # courses who successfully assigned added to the waiting list 
    waiting_list = [t[0] for t in assigned.keys()]
    
    # update counter
    count+=1


In [95]:
# dict of courses of each assigned TA
course={ta:courses for ta in waiting_list}
for ta in waiting_list:
    # setting max of TA teaching number of courses(max=2)
    if (ta in waiting_list)in range(1,2):
        # return count of each assigned TA
        countt = Counter([key[0] for key in assigned.keys()])
        for c in countt.keys():
            if countt[c] in range(1,2):
                #List of courses
                courr = course[ta]
                # Return course with the second most preferred for a specific TA
                cho = pref.loc[ta][pref.loc[ta].index.isin(courr)].sort_values().head(2).index[1]
                # Add to the dict the key(TA, course assigned) with value (rank in preferred, rank in skilled)
                assigned[(ta, cho)]=(pref.loc[ta][cho],
                                                 skill.loc[ta][cho])

# return count of each assigned course
coutt = Counter([key[1] for key in assigned.keys()])

    # cycle to choose the best options
    # loop on the repeated courses only
    # setting max of each course assigned twice
for c in coutt.keys():
    # setting max. no. of course assigning here
        if coutt[c] >(4):
            #return list of tuples after being sorted with their skilled rank and then remove the first tuple [0]
            # as being best choice to assign
            to_drop = sorted({pair: assigned[pair] for pair in assigned.keys() 
                    if c in pair}.items(), 
                   key=lambda x: x[1][1])[1:]
            
            for p_to_drop in to_drop:
                #delete the to_drop repeated assigning from assigned dict if it has least skilled compared to the preferred
                if (p_to_drop[1][1])>4:
                    del assigned[p_to_drop[0]]
                    
      

In [96]:

assigned = assigned.items()
assigned = sorted(assigned)
assigned

[(('Ahmed', '207'), (1, 3)),
 (('Ahmed', '479'), (2, 2)),
 (('Alaa', '207'), (2, 8)),
 (('Ali', '305'), (2, 8)),
 (('Ali', '307'), (1, 7)),
 (('Arwa', '307'), (1, 8)),
 (('Arwa', '379'), (2, 7)),
 (('Bassam', '305'), (1, 5)),
 (('Bassam', '317'), (2, 7)),
 (('Farah', '307'), (1, 1)),
 (('Mohammed', '207'), (1, 6)),
 (('Pola', '479'), (2, 6)),
 (('Salwa', '205'), (1, 1)),
 (('Salwa', '217'), (2, 3)),
 (('Yassmine', '307'), (2, 2))]